In [2]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re

In [3]:
df1 = pd.read_csv("C:/Users/mnkgr/OneDrive/Desktop/University Recommendation System/datasets/final_data_main.csv")
df1.shape

C:\Users\mnkgr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(485250, 21)

In [4]:
df1.drop(df1.columns[df1.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,University Of Waterloo,Systems Design Engineering,MS,NaN,Accepted,Website,"(1, 7, 2019)",1.561964e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(10, 7, 2019)",1562742000,NaN
1,Northeastern University,Electrical Engineering,PhD,F19,Rejected,Website,"(8, 7, 2019)",1.562569e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, 7, 2019)",1562742000,Got email today!
2,The University Of Auckland,Electrical And Electronic Engineering,MS,NaN,Accepted,Website,"(19, 6, 2019)",1.560928e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(9, 7, 2019)",1562655600,Top Student\nIELTS: 7.5\nContacted Supervisor\...
3,Radford University,Counseling Psychology PsyD.,Other,F19,Accepted,Phone,"(4, 3, 2019)",1.551686e+09,NaN,NaN,NaN,NaN,NaN,NaN,American,"(9, 7, 2019)",1562655600,NaN
4,University Of Chittagong,Computer Science,MS,NaN,NaN,Other,"(9, 7, 2019)",1.562656e+09,3.2,163.0,168.0,4.0,True,NaN,International,"(9, 7, 2019)",1562655600,NaN


In [5]:
df1.columns = ['universityName', 'major', 'program_name', 'intake', 'decision', 'applicationMethod', 'decdate', 'decdate_ts', 'cgpa', 'greVerbal', 'greQuant', 
           'greA', 'first_time_gre', 'gre_subject','status', 'post_data', 'post_timestamp', 'comments_on_application']
df1.head()

,universityName,major,program_name,intake,decision,applicationMethod,decdate,decdate_ts,cgpa,greVerbal,greQuant,greA,first_time_gre,gre_subject,status,post_data,post_timestamp,comments_on_application
0,University Of Waterloo,Systems Design Engineering,MS,NaN,Accepted,Website,"(1, 7, 2019)",1.561964e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(10, 7, 2019)",1562742000,NaN
1,Northeastern University,Electrical Engineering,PhD,F19,Rejected,Website,"(8, 7, 2019)",1.562569e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, 7, 2019)",1562742000,Got email today!
2,The University Of Auckland,Electrical And Electronic Engineering,MS,NaN,Accepted,Website,"(19, 6, 2019)",1.560928e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(9, 7, 2019)",1562655600,Top Student\nIELTS: 7.5\nContacted Supervisor\...
3,Radford University,Counseling Psychology PsyD.,Other,F19,Accepted,Phone,"(4, 3, 2019)",1.551686e+09,NaN,NaN,NaN,NaN,NaN,NaN,American,"(9, 7, 2019)",1562655600,NaN
4,University Of Chittagong,Computer Science,MS,NaN,NaN,Other,"(9, 7, 2019)",1.562656e+09,3.2,163.0,168.0,4.0,True,NaN,International,"(9, 7, 2019)",1562655600,NaN


In [6]:
df1 = df1[df1['decision'] =='Accepted']

df1.shape

(197058, 18)

In [7]:
df1 = df1[pd.notnull(df1['greQuant'])]
df1.shape

(61474, 18)

In [8]:
df1['greQuant'] = df1['greQuant'].fillna(130)
df1['greVerbal'] = df1['greVerbal'].fillna(130)
df1['greA'] = df1['greA'].fillna(0)
df1.greA.head()

7     4.0
14    3.0
17    5.5
28    4.0
46    4.0
Name: greA, dtype: float64

In [9]:
universityName = df1['universityName'].unique()

university_list = pd.DataFrame({'universityName':universityName})
university_list

,universityName
0,New York University (NYU) - Steinhardt
1,Ohio State University
2,Texas A&M University
3,St. Johns University
4,"University Of California, Irvine"
...,...
5316,Columbia University GSAS
5317,New York University(NYU)
5318,DeVry University
5319,Oregon University


In [10]:
df1.describe()

,decdate_ts,cgpa,greVerbal,greQuant,greA,gre_subject,post_timestamp
count,6.145400e+04,55589.000000,61474.000000,61474.000000,61474.000000,7175.000000,6.147400e+04
mean,1.431551e+09,3.715970,231.556333,248.826447,4.144757,796.411150,1.431763e+09
std,9.540728e+07,0.506153,174.575147,208.551820,1.111126,122.305977,8.079993e+07
min,-1.000000e+00,0.400000,130.000000,130.000000,0.000000,310.000000,1.263283e+09
25%,1.363244e+09,3.520000,155.000000,157.000000,3.500000,710.000000,1.363417e+09
50%,1.426662e+09,3.750000,161.000000,164.000000,4.000000,800.000000,1.427094e+09
75%,1.490771e+09,3.900000,167.000000,170.000000,5.000000,890.000000,1.491030e+09
max,1.360120e+10,9.990000,800.000000,800.000000,6.000000,990.000000,1.562569e+09


In [11]:
df1['greVerbal'].head()


7     157.0
14    150.0
17    157.0
28    140.0
46    155.0
Name: greVerbal, dtype: float64

In [13]:
def quantitative_score_conversion(quant_score):
    quantitative_score_list = []
    for old_quantitative_score in quant_score.tolist():
        if old_quantitative_score <= 170:
            quantitative_score_list.append(old_quantitative_score)
            continue
        else:
            old_quantitative_score = old_quantitative_score/4.7
            if old_quantitative_score <=130:
                quantitative_score_list.append(130)
            else:
                quantitative_score_list.append(old_quantitative_score)
    return quantitative_score_list

def verbal_score_conversion(verbal_score):
    verbal_score_list = []
    verbal_score_list = verbal_score.tolist()
    for old_verbal_score in verbal_score_list:
        if old_verbal_score <= 170:
            verbal_score_list.append(old_verbal_score)
            continue
        else:
            old_verbal_score = old_verbal_score/4.7
            if old_verbal_score <=130:
                verbal_score_list.append(130)
            else:
                verbal_score_list.append(old_verbal_score)
    return verbal_score_list

df1['greQuant'] = quantitative_score_conversion(df1['greQuant'])

df1['greVerbal'] = verbal_score_conversion(df1['greVerbal'])

MemoryError: 

In [ ]:
import seaborn as sns
sns.pairplot(data, palette="husl", x_vars=["greVerbal","cgpa","greQuant"], y_vars=["greVerbal","cgpa","greQuant"], height=8)
plt.show()


In [ ]:
df1 = df1.drop('major',1)
df1 = df1.drop('program_name',1)
df1 = df1.drop('intake',1)
df1 = df1.drop('decision',1)
df1 = df1.drop('applicaitonMethod',1)
df1 = df1.drop('decdate',1)
df1 = df1.drop('decdate_ts',1)
df1 = df1.drop('first_time_gre',1)
df1 = df1.drop('gre_subject',1)
df1 = df1.drop('status',1)
df1 = df1.drop('post_data',1)
df1 = df1.drop('post_timestamp',1)
df1 = df1.drop('comments_on_application',1)

list_of_university = list(set(df1['universityName'].tolist()))
for i in range(len(list_of_university)):
    if(len(df1[df1['universityName'] == list_of_university[i]]) < 100):
        df1 = df1[df1['universityName'] != list_of_university[i]]
df1 = df1.dropna()

df1.head()

In [ ]:
final_preprocessed_data = df1[['greVerbal', 'greQuant', 'greA', 'cgpa', 'universityName']]
final_preprocessed_data.head()

final_preprocessed_data.to_csv('./data/csv/Processed_data.csv')

In [ ]:
import math
from sklearn import neighbors, datasets
from numpy.random import permutation
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support





In [ ]:
#similar_univs = pandas.read_csv('similar_universities.csv')
random_indices = permutation(data.index)
test_cutoff = math.floor(len(data)/5)
print(test_cutoff)
test = processed_data.loc[random_indices[1:test_cutoff]]
train = processed_data.loc[random_indices[test_cutoff:]]
train_output_data = train['univName']
print("train Output data", train_output_data)
train_input_data = train
train_input_data = train_input_data.drop('univName',1)
print("train input data", train_input_data)
test_output_data = test['univName']
print("test Output data", test_output_data)
test_input_data = test
test_input_data = test_input_data.drop('univName',1)
print("test input data", test_input_data)

In [ ]:

def euclideanDistance(data1, data2, length):
    distance = 0
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

def knn(trainingSet, testInstance, k):
    print(k)
    distances = {}
    sort = {}
    length = testInstance.shape[1]

    for x in range(len(trainingSet)):

        dist = euclideanDistance(testInstance, trainingSet.iloc[x], length)
        
        distances[x] = dist[0]

    sorted_d = sorted(distances.items(), key=lambda x: x[1])

 
    neighbors = []

    for x in range(k):
        neighbors.append(sorted_d[x][0])

    classVotes = {}

    for x in range(len(neighbors)):
        response = trainingSet.iloc[neighbors[x]][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1

    sortedVotes = sorted(classVotes.items(), key=lambda x: x[1], reverse=True)
 
    return(sortedVotes, neighbors)


In [ ]:

testSet = [[142, 153, 5.0, 3.6]]
test = pd.DataFrame(testSet)
test.shape

In [ ]:

k = 7

result,neigh= knn(processed_data, test, k)


list1 = []
list2 = []
for i in result:
    list1.append(i[0])
    list2.append(i[1])
for i in list1:
    print(i)
    


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(processed_data.iloc[:,0:4], data['univName'])

print(neigh.predict(test))